In [10]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping


batch_size =64
epochs = 5
#load data
(x_train,y_train),(x_test,y_test)=mnist.load_data()

#prepare dataset
img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]

x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

input_shape = (img_rows,img_cols,1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_class = y_test.shape[1]


#create model
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=input_shape,activation='relu'))
model.add(Conv2D(64,(3,3),input_shape=input_shape,activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer = 'Adam',
             metrics=['accuracy'])

checkpoint = ModelCheckpoint("MNIST_Checkpoint.h5",monitor='val_loss',
                            mode='min',save_best_only=True,verbose=1)

callbacks = [checkpoint]
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               4718720   
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

In [6]:
history = model.fit(x_train,y_train,batch_size=batch_size,
                   epochs=epochs,verbose=2,validation_data=(x_test,y_test),
                   callbacks=callbacks)
score = model.evaluate(x_test,y_test)

print('test loss = %f and test_val = %f'%(score[0],score[1]))

Epoch 1/5

Epoch 00001: val_loss improved from inf to 0.04603, saving model to MNIST_Checkpoint.h5
938/938 - 10s - loss: 0.1847 - accuracy: 0.9438 - val_loss: 0.0460 - val_accuracy: 0.9848
Epoch 2/5

Epoch 00002: val_loss improved from 0.04603 to 0.03693, saving model to MNIST_Checkpoint.h5
938/938 - 10s - loss: 0.0742 - accuracy: 0.9775 - val_loss: 0.0369 - val_accuracy: 0.9870
Epoch 3/5

Epoch 00003: val_loss improved from 0.03693 to 0.03277, saving model to MNIST_Checkpoint.h5
938/938 - 10s - loss: 0.0527 - accuracy: 0.9836 - val_loss: 0.0328 - val_accuracy: 0.9887
Epoch 4/5

Epoch 00004: val_loss improved from 0.03277 to 0.02939, saving model to MNIST_Checkpoint.h5
938/938 - 10s - loss: 0.0413 - accuracy: 0.9874 - val_loss: 0.0294 - val_accuracy: 0.9906
Epoch 5/5

Epoch 00005: val_loss did not improve from 0.02939
938/938 - 10s - loss: 0.0334 - accuracy: 0.9893 - val_loss: 0.0296 - val_accuracy: 0.9907
313/313 [==============================] - 1s 3ms/step - loss: 0.0296 - accuracy

# Add Early stopping to callbacks

In [9]:
earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True)

callbacks = [earlystop,checkpoint]

In [ ]:
history = model.fit(x_train,y_train,
                   batch_size=batch_size,
                   epochs=epochs,
                   callbacks=callbacks,
                   verbose=2,
                   validation_data=(x_test,y_test))

score = model.evaluate(x_test,y_test)

print('test loss = %f and test_val = %f'%(score[0],score[1]))

Epoch 1/5

Epoch 00001: val_loss improved from inf to 0.04761, saving model to MNIST_Checkpoint.h5
938/938 - 10s - loss: 0.1858 - accuracy: 0.9442 - val_loss: 0.0476 - val_accuracy: 0.9837
Epoch 2/5

Epoch 00002: val_loss improved from 0.04761 to 0.03451, saving model to MNIST_Checkpoint.h5
938/938 - 10s - loss: 0.0752 - accuracy: 0.9774 - val_loss: 0.0345 - val_accuracy: 0.9882
Epoch 3/5

Epoch 00003: val_loss improved from 0.03451 to 0.03210, saving model to MNIST_Checkpoint.h5
938/938 - 11s - loss: 0.0537 - accuracy: 0.9835 - val_loss: 0.0321 - val_accuracy: 0.9894
Epoch 4/5

Epoch 00004: val_loss improved from 0.03210 to 0.03186, saving model to MNIST_Checkpoint.h5
938/938 - 10s - loss: 0.0407 - accuracy: 0.9870 - val_loss: 0.0319 - val_accuracy: 0.9903
Epoch 5/5


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)